# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 2 2022 12:00AM,238578,10,0085732383,ISDIN Corporation,Released
1,Jun 2 2022 12:00AM,238578,10,0085732384,ISDIN Corporation,Released
2,Jun 2 2022 12:00AM,238578,10,0085732439,ISDIN Corporation,Released
3,Jun 2 2022 12:00AM,238578,10,0085732470,ISDIN Corporation,Released
4,Jun 2 2022 12:00AM,238578,10,0085732523,ISDIN Corporation,Released
5,Jun 2 2022 12:00AM,238578,10,0085732535,ISDIN Corporation,Released
6,Jun 2 2022 12:00AM,238578,10,0085732547,ISDIN Corporation,Released
7,Jun 2 2022 12:00AM,238578,10,0085732603,ISDIN Corporation,Released
8,Jun 2 2022 12:00AM,238578,10,0085732624,ISDIN Corporation,Released
9,Jun 2 2022 12:00AM,238578,10,0085732633,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,238587,Released,1
37,238588,Executing,1
38,238590,Released,1
39,238591,Released,4
40,238593,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238587,NaN,NaN,1.0
238588,NaN,1.0,NaN
238590,NaN,NaN,1.0
238591,NaN,NaN,4.0
238593,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238489,0.0,0.0,1.0
238490,0.0,21.0,1.0
238501,14.0,3.0,2.0
238515,0.0,0.0,1.0
238516,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238489,0,0,1
238490,0,21,1
238501,14,3,2
238515,0,0,1
238516,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238489,0,0,1
1,238490,0,21,1
2,238501,14,3,2
3,238515,0,0,1
4,238516,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238489,,,1
1,238490,,21,1
2,238501,14,3,2
3,238515,,,1
4,238516,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 2 2022 12:00AM,238578,10,ISDIN Corporation
14,Jun 2 2022 12:00AM,238585,10,"E5 Pharma, LLC"
26,Jun 2 2022 12:00AM,238591,10,Eywa Pharma Inc.
30,Jun 1 2022 4:36PM,238593,20,Sartorius Stedim North America
31,Jun 1 2022 4:28PM,238590,21,"NBTY Global, Inc."
32,Jun 1 2022 4:18PM,238588,21,"NBTY Global, Inc."
33,Jun 1 2022 4:18PM,238586,19,"Granules Pharmaceuticals, Inc."
34,Jun 1 2022 4:18PM,238587,20,Sartorius Stedim North America
35,Jun 1 2022 3:59PM,238583,20,Sartorius Stedim North America
36,Jun 1 2022 3:48PM,238581,20,Sartorius Stedim North America


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 2 2022 12:00AM,238578,10,ISDIN Corporation,,,14
1,Jun 2 2022 12:00AM,238585,10,"E5 Pharma, LLC",,5,7
2,Jun 2 2022 12:00AM,238591,10,Eywa Pharma Inc.,,,4
3,Jun 1 2022 4:36PM,238593,20,Sartorius Stedim North America,,,1
4,Jun 1 2022 4:28PM,238590,21,"NBTY Global, Inc.",,,1
5,Jun 1 2022 4:18PM,238588,21,"NBTY Global, Inc.",,1,
6,Jun 1 2022 4:18PM,238586,19,"Granules Pharmaceuticals, Inc.",,,1
7,Jun 1 2022 4:18PM,238587,20,Sartorius Stedim North America,,,1
8,Jun 1 2022 3:59PM,238583,20,Sartorius Stedim North America,,,1
9,Jun 1 2022 3:48PM,238581,20,Sartorius Stedim North America,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 12:00AM,238578,10,ISDIN Corporation,14,,
1,Jun 2 2022 12:00AM,238585,10,"E5 Pharma, LLC",7,5,
2,Jun 2 2022 12:00AM,238591,10,Eywa Pharma Inc.,4,,
3,Jun 1 2022 4:36PM,238593,20,Sartorius Stedim North America,1,,
4,Jun 1 2022 4:28PM,238590,21,"NBTY Global, Inc.",1,,
5,Jun 1 2022 4:18PM,238588,21,"NBTY Global, Inc.",,1,
6,Jun 1 2022 4:18PM,238586,19,"Granules Pharmaceuticals, Inc.",1,,
7,Jun 1 2022 4:18PM,238587,20,Sartorius Stedim North America,1,,
8,Jun 1 2022 3:59PM,238583,20,Sartorius Stedim North America,1,,
9,Jun 1 2022 3:48PM,238581,20,Sartorius Stedim North America,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 12:00AM,238578,10,ISDIN Corporation,14,,
1,Jun 2 2022 12:00AM,238585,10,"E5 Pharma, LLC",7,5,
2,Jun 2 2022 12:00AM,238591,10,Eywa Pharma Inc.,4,,
3,Jun 1 2022 4:36PM,238593,20,Sartorius Stedim North America,1,,
4,Jun 1 2022 4:28PM,238590,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 12:00AM,238578,10,ISDIN Corporation,14.0,NaN,NaN
1,Jun 2 2022 12:00AM,238585,10,"E5 Pharma, LLC",7.0,5.0,NaN
2,Jun 2 2022 12:00AM,238591,10,Eywa Pharma Inc.,4.0,NaN,NaN
3,Jun 1 2022 4:36PM,238593,20,Sartorius Stedim North America,1.0,NaN,NaN
4,Jun 1 2022 4:28PM,238590,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 12:00AM,238578,10,ISDIN Corporation,14.0,0.0,0.0
1,Jun 2 2022 12:00AM,238585,10,"E5 Pharma, LLC",7.0,5.0,0.0
2,Jun 2 2022 12:00AM,238591,10,Eywa Pharma Inc.,4.0,0.0,0.0
3,Jun 1 2022 4:36PM,238593,20,Sartorius Stedim North America,1.0,0.0,0.0
4,Jun 1 2022 4:28PM,238590,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1431259,29.0,8.0,14.0
12,715642,67.0,0.0,0.0
15,238490,1.0,21.0,0.0
16,954141,2.0,2.0,0.0
18,477097,2.0,0.0,0.0
19,1192778,5.0,0.0,0.0
20,3339941,34.0,0.0,0.0
21,477178,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1431259,29.0,8.0,14.0
1,12,715642,67.0,0.0,0.0
2,15,238490,1.0,21.0,0.0
3,16,954141,2.0,2.0,0.0
4,18,477097,2.0,0.0,0.0
5,19,1192778,5.0,0.0,0.0
6,20,3339941,34.0,0.0,0.0
7,21,477178,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,29.0,8.0,14.0
1,12,67.0,0.0,0.0
2,15,1.0,21.0,0.0
3,16,2.0,2.0,0.0
4,18,2.0,0.0,0.0
5,19,5.0,0.0,0.0
6,20,34.0,0.0,0.0
7,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,29.0
1,12,Released,67.0
2,15,Released,1.0
3,16,Released,2.0
4,18,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Completed,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,8.0,0.0,21.0,2.0,0.0,0.0,0.0,1.0
Released,29.0,67.0,1.0,2.0,2.0,5.0,34.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Completed,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,8.0,0.0,21.0,2.0,0.0,0.0,0.0,1.0
2,Released,29.0,67.0,1.0,2.0,2.0,5.0,34.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Completed,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,8.0,0.0,21.0,2.0,0.0,0.0,0.0,1.0
2,Released,29.0,67.0,1.0,2.0,2.0,5.0,34.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()